<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019/blob/master/Packages/Git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from google.colab import drive, files
from urllib.parse import urlsplit
from pathlib import Path
import json

# Configure

In [0]:
def config(confile="", options="--global"):
  """
  Configure Git
  confile: path to .json configuration file, containing text: {"email": "<email>", "name": "<display name>", ...}
  options: supported by git-config (https://git-scm.com/docs/git-config)
  """

  # Configurations
  confile = confile or input("Enter path to .json configuration file: ")
  with open(confile) as f:
    configs = json.load(f)

  # Configure
  !git config {options} user.email "{configs['email']}"
  !git config {options} user.name "{configs['name']}"

if __name__ == "__main__":
  config()

Enter path to .json configuration file: /drive/My Drive/GitHub/config.json


# Clone

In [3]:
def clone(url, dest=".", name="", authfile="", options="--single-branch -b master", reloc=True):
  """
  Clone url into dest
  name: if provided, rename repository
  authfile: path to .json authentication file, containing text: {"username": "<username>", "password": "<password>"}, for private remote
  options: supported by git-clone (https://git-scm.com/docs/git-clone)
  reloc: if True, relocate to repository
  """

  rurl = urlsplit(url)
  dest = Path(dest).resolve()
  repo = dest / (name or Path(rurl.path).name)

  # Nested repositories not allowed
  out = !git -C "{dest}" rev-parse
  if not out: # inside repository
    raise ValueError("Can't clone into existing repository")
  
  # Construct authenticated remote
  rauth = rurl
  if authfile:
    # Authentication information
    with open(authfile) as f:
      auth = json.load(f)
    # Insert username and password
    rauth = rurl._replace(netloc=f"{auth['username']}:{auth['password']}@{rurl.hostname}")

  # Clone
  !git clone {options} "{rauth.geturl()}" "{repo}"
  
  # Relocate
  if reloc:
    os.chdir(repo)

  return repo

if __name__ == "__main__":
  url = "https://github.com/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit"
  clone(url)

Cloning into '/content/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 1286 (delta 50), reused 0 (delta 0), pack-reused 1182
Receiving objects: 100% (1286/1286), 3.98 MiB | 12.35 MiB/s, done.
Resolving deltas: 100% (656/656), done.


# Push

In [0]:
def push(url, branch="HEAD", authfile=""):
  """
  Push branch to url
  branch: defaults to current branch; if not provided, all branches
  authfile: path to .json authentication file, containing text: {"username": "<username>", "password": "<password>"}
  """
  
  rurl = urlsplit(url)

  # Authentication information
  authfile = authfile or input("Enter path to .json authentication file: ")
  with open(authfile) as f:
    auth = json.load(f)

  # Construct authenticated remote
  rauth = rurl._replace(netloc=f"{auth['username']}:{auth['password']}@{rurl.hostname}") # add username and password
  
  # Push
  if branch:
    !git push "{rauth.geturl()}" "{branch}"
  else:
    !git push --all "{rauth.geturl()}"

if __name__ == "__main__":
  url = "https://github.com/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit"
  push(url)

KeyboardInterrupt: ignored